In [ ]:
# import libraries
import requests
import pandas as pd
import time

In [ ]:
# Keys
API_KEY = "xxxxxx"
CHANNEL_ID = "xxxxxx"

In [ ]:
def get_video_details(video_id):

    # collecting view, like, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id=" + video_id + "&part=statistics&key=" + API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, comment_count

In [ ]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key=" + API_KEY + "&channelId=" + CHANNEL_ID + "&part=snippet,id&order=date&maxResults=10000&" + pageToken

        response = requests.get(url).json()
        # give it a second before starting the for loop
        time.sleep(1) 
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            # if none, it means it reached the last page and break out of it
            if response['nextPageToken'] != None: 
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [ ]:
# main

# build our dataframe
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"])

df = get_videos(df)
df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,9oNY9EV3MiM,Speed up your Python Data Manipulation with La...,2023-02-28,438,12,0
1,YN-5NQO4iSU,Solving Amazon Interview Questions with Lambda...,2023-02-22,585,29,2
2,ltzJhIs8voQ,Mastering SQL String Manipulation with Regex_s...,2023-02-21,311,22,0
3,BgN5hpl3WKc,Solving a Tricky Google SQL Question with Stri...,2023-02-16,823,35,4
4,GxR66KdqBFU,Analyzing Datasets with Window Functions: Firs...,2023-02-16,394,12,0
...,...,...,...,...,...,...
134,UX4_IgagL9I,How to Use Google Colaboratory | Import a CSV ...,2020-05-02,2511,14,2
135,tDdo3FiWpgE,Interview Questions for SQL Joins and Subqueries,2020-02-01,1904,28,0
136,wW827gqxlRY,SQL Job Interview Mistakes #2,2019-09-15,1171,21,0
137,xbc2GpGUXwc,SQL Job Interview Mistakes #1,2019-08-03,8137,76,1


In [ ]:
df.to_csv('youtube_vids_1st_pull.csv')